In [1]:
%pwd

'/workspaces/-renci_perfonmans_de-erlendirme_MLOPS_Project/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/workspaces/-renci_perfonmans_de-erlendirme_MLOPS_Project'

# Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen = True)
class DataingestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [5]:
from src.ogrenci_perfonmans_degerlendirme.constants import *
from src.ogrenci_perfonmans_degerlendirme.utils.common import read_yaml,create_directories

In [6]:
class ConfigManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataingestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataingestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


# Componenets

In [7]:
import os
import urllib.request as request
import zipfile
from src.ogrenci_perfonmans_degerlendirme import logger
from src.ogrenci_perfonmans_degerlendirme.utils.common import get_size
import pandas as pd

In [8]:
class DataIngestion:
    def __init__(self, config:DataingestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} indirildi. Ve dosyanın bilgileri şunlardır: \n{headers}")
        else:
            logger.info(f'Böyle bir dosya zaten mevcut {get_size(Path(self.config.local_data_file))}')   
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file) as zip_ref:
            zip_ref.extractall(unzip_path)

# Pipeline

In [9]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-18 14:13:31,130: INFO: 2626022347: artifacts/data_ingestion/data.zip indirildi. Ve dosyanın bilgileri şunlardır: 
X-GUploader-UploadID: AFIdbgQThdkPCTpse661C-6haY36P9AJmB0seu6KVWyx45qRQS_POMyZIUbFPEYypIDaW7jPcfVwBMw
Expires: Sat, 18 Jan 2025 14:13:31 GMT
Date: Sat, 18 Jan 2025 14:13:31 GMT
Cache-Control: private, max-age=0
Last-Modified: Thu, 07 Mar 2024 06:14:12 GMT
ETag: "47d2ff7dbfc7997cc2cf67cb80e41cdd"
x-goog-generation: 1709792052142033
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 8907
Content-Type: application/zip
x-goog-hash: crc32c=A49eQg==
x-goog-hash: md5=R9L/fb/HmXzCz2fLgOQc3Q==
x-goog-storage-class: MULTI_REGIONAL
Accept-Ranges: bytes
Content-Length: 8907
Access-Control-Allow-Origin: *
Access-Control-Expose-Headers: Content-Type
Access-Control-Expose-Headers: User-Agent
Server: UploadServer
Alt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000
Connection: close

]
